In [2]:
import scipy.io as sio
import numpy as np
import os
import gc
import six.moves.urllib as urllib
import cv2
import time
import xml.etree.cElementTree as ET
import random
import shutil as sh
from shutil import copyfile
import zipfile

import csv

In [3]:

def save_csv(csv_path, csv_content):
    with open(csv_path, 'w') as csvfile:
        wr = csv.writer(csvfile)
        for i in range(len(csv_content)):
            wr.writerow(csv_content[i])


def get_bbox_visualize(base_path, dir):
    image_path_array = []
    for root, dirs, filenames in os.walk(base_path + dir):
        for f in filenames:
            if(f.split(".")[1] == "jpg"):
                img_path = base_path + dir + "/" + f
                image_path_array.append(img_path)

    #sort image_path_array to ensure its in the low to high order expected in polygon.mat
    image_path_array.sort()
    boxes = sio.loadmat(
        base_path + dir + "/polygons.mat")
    # there are 100 of these per folder in the egohands dataset
    polygons = boxes["polygons"][0]
    # first = polygons[0]
    # print(len(first))
    pointindex = 0

    for first in polygons:
        index = 0

        font = cv2.FONT_HERSHEY_SIMPLEX

        img_id = image_path_array[pointindex]
        img = cv2.imread(img_id)

        img_params = {}
        img_params["width"] = np.size(img, 1)
        img_params["height"] = np.size(img, 0)
        head, tail = os.path.split(img_id)
        img_params["filename"] = tail
        img_params["path"] = os.path.abspath(img_id)
        img_params["type"] = "train"
        pointindex += 1

        boxarray = []
        csvholder = []
        for pointlist in first:
            pst = np.empty((0, 2), int)
            max_x = max_y = min_x = min_y = height = width = 0

            findex = 0
            for point in pointlist:
                if(len(point) == 2):
                    x = int(point[0])
                    y = int(point[1])

                    if(findex == 0):
                        min_x = x
                        min_y = y
                    findex += 1
                    max_x = x if (x > max_x) else max_x
                    min_x = x if (x < min_x) else min_x
                    max_y = y if (y > max_y) else max_y
                    min_y = y if (y < min_y) else min_y
                    # print(index, "====", len(point))
                    appeno = np.array([[x, y]])
                    pst = np.append(pst, appeno, axis=0)
                    cv2.putText(img, ".", (x, y), font, 0.7,
                                (255, 255, 255), 2, cv2.LINE_AA)

            hold = {}
            hold['minx'] = min_x
            hold['miny'] = min_y
            hold['maxx'] = max_x
            hold['maxy'] = max_y
            if (min_x > 0 and min_y > 0 and max_x > 0 and max_y > 0):
                boxarray.append(hold)
                labelrow = [tail,
                            np.size(img, 1), np.size(img, 0), "hand", min_x, min_y, max_x, max_y]
                csvholder.append(labelrow)

            cv2.polylines(img, [pst], True, (0, 255, 255), 1)
            cv2.rectangle(img, (min_x, max_y),
                          (max_x, min_y), (0, 255, 0), 1)

        csv_path = img_id.split(".")[0]
        if not os.path.exists(csv_path + ".csv"):
            cv2.putText(img, "DIR : " + dir + " - " + tail, (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (77, 255, 9), 2)
            cv2.imshow('Verifying annotation ', img)
            save_csv(csv_path + ".csv", csvholder)
            print("===== saving csv file for ", tail)
        cv2.waitKey(2)  # close window when a key press is detected


def create_directory(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# combine all individual csv files for each image into a single csv file per folder.


def generate_label_files(image_dir):
    header = ['filename', 'width', 'height',
              'class', 'xmin', 'ymin', 'xmax', 'ymax']
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            csvholder = []
            csvholder.append(header)
            loop_index = 0
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "csv"):
                    loop_index += 1
                    #print(loop_index, f)
                    csv_file = open(image_dir + dir + "/" + f, 'r')
                    reader = csv.reader(csv_file)
                    for row in reader:
                        csvholder.append(row)
                    csv_file.close()
                    os.remove(image_dir + dir + "/" + f)
            save_csv(image_dir + dir + "/" + dir + "_labels.csv", csvholder)
            print("Saved label csv for ", dir, image_dir +
                  dir + "/" + dir + "_labels.csv")


# Split data, copy to train/test folders
def split_data_test_eval_train(image_dir):
    create_directory("images")
    create_directory("images/train")
    create_directory("images/test")

    data_size = 4000
    loop_index = 0
    data_sampsize = int(0.1 * data_size)
    test_samp_array = random.sample(range(data_size), k=data_sampsize)

    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "jpg"):
                    loop_index += 1
                    print(loop_index, f)

                    if loop_index in test_samp_array:
                        os.rename(image_dir + dir +
                                  "/" + f, "images/test/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "images/test/" + f.split(".")[0] + ".csv")
                    else:
                        os.rename(image_dir + dir +
                                  "/" + f, "images/train/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "images/train/" + f.split(".")[0] + ".csv")
                    print(loop_index, image_dir + f)
            print(">   done scanning director ", dir)
            os.remove(image_dir + dir + "/polygons.mat")
            os.rmdir(image_dir + dir)

        print("Train/test content generation complete!")
        generate_label_files("images/")


def generate_csv_files(image_dir):
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            get_bbox_visualize(image_dir, dir)

    print("CSV generation complete!\nGenerating train/test/eval folders")
    split_data_test_eval_train("egohands/_LABELLED_SAMPLES/")


# rename image files so we can have them all in a train/test/eval folder.
def rename_files(image_dir):
    print("Renaming files")
    loop_index = 0
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if (dir not in f):
                    if(f.split(".")[1] == "jpg"):
                        loop_index += 1
                        os.rename(image_dir + dir +
                                  "/" + f, image_dir + dir +
                                  "/" + dir + "_" + f)
                else:
                    break

    generate_csv_files("egohands/_LABELLED_SAMPLES/")

def extract_folder(dataset_path):
    print("Egohands dataset already downloaded.\nGenerating CSV files")
    if not os.path.exists("egohands"):
        zip_ref = zipfile.ZipFile(dataset_path, 'r')
        print("> Extracting Dataset files")
        zip_ref.extractall("egohands")
        print("> Extraction complete")
        zip_ref.close()
        rename_files("egohands/_LABELLED_SAMPLES/")

def download_egohands_dataset(dataset_url, dataset_path):
    is_downloaded = os.path.exists(dataset_path)
    if not is_downloaded:
        print(
            "> downloading egohands dataset. This may take a while (1.3GB, say 3-5mins). Coffee break?")
        opener = urllib.request.URLopener()
        opener.retrieve(dataset_url, dataset_path)
        print("> download complete")
        extract_folder(dataset_path);

    else:
        extract_folder(dataset_path)


EGOHANDS_DATASET_URL = "http://vision.soic.indiana.edu/egohands_files/egohands_data.zip"
EGO_HANDS_FILE = "egohands_data.zip"


download_egohands_dataset(EGOHANDS_DATASET_URL, EGO_HANDS_FILE)

> downloading egohands dataset. This may take a while (1.3GB, say 3-5mins). Coffee break?


C:\Users\User-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:217: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


> download complete
Egohands dataset already downloaded.
Generating CSV files
> Extracting Dataset files
> Extraction complete
Renaming files
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0011.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0036.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0099.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0113.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0176.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0183.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0184.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0187.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0189.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0206.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0229.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0241.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame_0295.jpg
===== saving csv file for  CARDS_COURTYARD_B_T_frame

===== saving csv file for  CARDS_COURTYARD_H_S_frame_0861.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_0958.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_0994.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1000.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1002.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1021.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1024.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1026.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1036.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1074.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1115.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1135.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1185.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1195.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1232.jpg
===== saving csv file for  CARDS_COURTYARD_H_S_frame_1245.jpg
===== sa

===== saving csv file for  CARDS_COURTYARD_S_H_frame_1815.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_1913.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_1951.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_1991.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_1993.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2000.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2027.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2090.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2098.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2108.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2138.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2148.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2154.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2165.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2183.jpg
===== saving csv file for  CARDS_COURTYARD_S_H_frame_2186.jpg
===== sa

...

4420 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0472.jpg
4421 PUZZLE_OFFICE_B_H_frame_0502.jpg
4421 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0502.jpg
4422 PUZZLE_OFFICE_B_H_frame_0516.jpg
4422 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0516.jpg
4423 PUZZLE_OFFICE_B_H_frame_0564.jpg
4423 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0564.jpg
4424 PUZZLE_OFFICE_B_H_frame_0573.jpg
4424 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0573.jpg
4425 PUZZLE_OFFICE_B_H_frame_0655.jpg
4425 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0655.jpg
4426 PUZZLE_OFFICE_B_H_frame_0712.jpg
4426 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0712.jpg
4427 PUZZLE_OFFICE_B_H_frame_0715.jpg
4427 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0715.jpg
4428 PUZZLE_OFFICE_B_H_frame_0752.jpg
4428 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0752.jpg
4429 PUZZLE_OFFICE_B_H_frame_0763.jpg
4429 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0763.jpg
4430 PUZ

4504 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0082.jpg
4505 PUZZLE_OFFICE_H_B_frame_0092.jpg
4505 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0092.jpg
4506 PUZZLE_OFFICE_H_B_frame_0109.jpg
4506 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0109.jpg
4507 PUZZLE_OFFICE_H_B_frame_0127.jpg
4507 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0127.jpg
4508 PUZZLE_OFFICE_H_B_frame_0234.jpg
4508 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0234.jpg
4509 PUZZLE_OFFICE_H_B_frame_0359.jpg
4509 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0359.jpg
4510 PUZZLE_OFFICE_H_B_frame_0379.jpg
4510 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0379.jpg
4511 PUZZLE_OFFICE_H_B_frame_0384.jpg
4511 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0384.jpg
4512 PUZZLE_OFFICE_H_B_frame_0469.jpg
4512 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0469.jpg
4513 PUZZLE_OFFICE_H_B_frame_0496.jpg
4513 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_0496.jpg
4514 PUZ

4591 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2466.jpg
4592 PUZZLE_OFFICE_H_B_frame_2470.jpg
4592 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2470.jpg
4593 PUZZLE_OFFICE_H_B_frame_2501.jpg
4593 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2501.jpg
4594 PUZZLE_OFFICE_H_B_frame_2508.jpg
4594 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2508.jpg
4595 PUZZLE_OFFICE_H_B_frame_2570.jpg
4595 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2570.jpg
4596 PUZZLE_OFFICE_H_B_frame_2599.jpg
4596 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2599.jpg
4597 PUZZLE_OFFICE_H_B_frame_2623.jpg
4597 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2623.jpg
4598 PUZZLE_OFFICE_H_B_frame_2628.jpg
4598 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2628.jpg
4599 PUZZLE_OFFICE_H_B_frame_2671.jpg
4599 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2671.jpg
4600 PUZZLE_OFFICE_H_B_frame_2687.jpg
4600 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B_frame_2687.jpg
>   done

4676 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_1960.jpg
4677 PUZZLE_OFFICE_S_T_frame_1973.jpg
4677 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_1973.jpg
4678 PUZZLE_OFFICE_S_T_frame_2000.jpg
4678 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2000.jpg
4679 PUZZLE_OFFICE_S_T_frame_2029.jpg
4679 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2029.jpg
4680 PUZZLE_OFFICE_S_T_frame_2072.jpg
4680 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2072.jpg
4681 PUZZLE_OFFICE_S_T_frame_2110.jpg
4681 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2110.jpg
4682 PUZZLE_OFFICE_S_T_frame_2116.jpg
4682 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2116.jpg
4683 PUZZLE_OFFICE_S_T_frame_2183.jpg
4683 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2183.jpg
4684 PUZZLE_OFFICE_S_T_frame_2243.jpg
4684 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2243.jpg
4685 PUZZLE_OFFICE_S_T_frame_2254.jpg
4685 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T_frame_2254.jpg
4686 PUZ

4761 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1765.jpg
4762 PUZZLE_OFFICE_T_S_frame_1787.jpg
4762 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1787.jpg
4763 PUZZLE_OFFICE_T_S_frame_1805.jpg
4763 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1805.jpg
4764 PUZZLE_OFFICE_T_S_frame_1821.jpg
4764 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1821.jpg
4765 PUZZLE_OFFICE_T_S_frame_1830.jpg
4765 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1830.jpg
4766 PUZZLE_OFFICE_T_S_frame_1846.jpg
4766 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1846.jpg
4767 PUZZLE_OFFICE_T_S_frame_1895.jpg
4767 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1895.jpg
4768 PUZZLE_OFFICE_T_S_frame_1900.jpg
4768 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1900.jpg
4769 PUZZLE_OFFICE_T_S_frame_1976.jpg
4769 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1976.jpg
4770 PUZZLE_OFFICE_T_S_frame_1985.jpg
4770 egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1985.jpg
4771 PUZ